# Tanzania Water Well Data Cleaning

In this project I work with the Tanzania Water Well data.

We are given the following prompt with this data set:

`Tanzania, as a developing country, struggles with providing clean water to its population of over 57,000,000. There are many water points already established in the country, but some are in need of repair while others have failed altogether.`

`Build a classifier to predict the condition of a water well, using information about the sort of pump, when it was installed, etc. Your audience could be an NGO focused on locating wells needing repair, or the Government of Tanzania looking to find patterns in non-functional wells to influence how new wells are built. Note that this is a ternary classification problem by default, but can be engineered to be binary.`

This notebook contains the data cleaning process for the training data. As we clean, let's also make note of ideas for pre-processing in the modeling stage.

#### Imports

In [1]:
import pandas as pd
import numpy as np

#### Targets

Let's look at our targets first.

In [2]:
with open('../data/Tanzania_Training_Merged.csv') as f:
    df = pd.read_csv(f)

# Increase columns displayed so we can see    
pd.set_option('display.max_columns', 50)

In [3]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,0
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1


### Let's Get Cleaning

The plan is to go column by column and decide the following:
 - Should we keep this column?
 - Are there null values we need to handle?
 - How will we address this feature in pre-processing for the model?
     - If categorical, will likely need to encode
     - If numerical, will need to scale
     
We will be referencing this data dictionary, provided [here](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/#the-features-in-this-dataset) with the dataset. 

- amount_tsh - Total static head (amount water available to waterpoint)
- date_recorded - The date the row was entered
- funder - Who funded the well
- gps_height - Altitude of the well
- installer - Organization that installed the well
- longitude - GPS coordinate
- latitude - GPS coordinate
- wpt_name - Name of the waterpoint if there is one
- num_private -
- basin - Geographic water basin
- subvillage - Geographic location
- region - Geographic location
- region_code - Geographic location (coded)
- district_code - Geographic location (coded)
- lga - Geographic location
- ward - Geographic location
- population - Population around the well
- public_meeting - True/False
- recorded_by - Group entering this row of data
- scheme_management - Who operates the waterpoint
- scheme_name - Who operates the waterpoint
- permit - If the waterpoint is permitted
- construction_year - Year the waterpoint was constructed
- extraction_type - The kind of extraction the waterpoint uses
- extraction_type_group - The kind of extraction the waterpoint uses
- extraction_type_class - The kind of extraction the waterpoint uses
- management - How the waterpoint is managed
- management_group - How the waterpoint is managed
- payment - What the water costs
- payment_type - What the water costs
- water_quality - The quality of the water
- quality_group - The quality of the water
- quantity - The quantity of water
- quantity_group - The quantity of water
- source - The source of the water
- source_type - The source of the water
- source_class - The source of the water
- waterpoint_type - The kind of waterpoint
- waterpoint_type_group - The kind of waterpoint
     
Here we go!

#### id

In [4]:
df.id.value_counts(dropna=False)

2047     1
72310    1
49805    1
51852    1
62091    1
        ..
46396    1
36155    1
34106    1
38200    1
0        1
Name: id, Length: 59400, dtype: int64

So we have one id per record. That adds up. We can remove this feature as the id number should have no bearing on well function.

In [5]:
# Drop ID
df = df.drop('id', axis=1)

#### amount_tsh

In [6]:
df.amount_tsh.describe()

count     59400.000000
mean        317.650385
std        2997.574558
min           0.000000
25%           0.000000
50%           0.000000
75%          20.000000
max      350000.000000
Name: amount_tsh, dtype: float64

In [7]:
df.amount_tsh.value_counts(dropna=False)

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64

Not normally distributed. We'll definitely want to scale this. Let's start a running list of numerical and categorical columns that we are keeping so we can add to them as we go.

We also note a significant number of 0 values here. We're going to assume these are actually missing values and replace them with NaN's.

In [8]:
# Create empty lists
num_columns = []
cat_columns = []
bin_columns = []
drop_columns = []

# Amount_tsh will go into numerical column list
num_columns.append('amount_tsh')

# Replace 0 with NaN
df = df.replace({'amount_tsh': {0: np.nan}})

#### date_recorded

In [9]:
df.date_recorded.max()

'2013-12-03'

In [10]:
df.date_recorded.min()

'2002-10-14'

So these records have been recorded between 2002 and 2013. That's nice to know, but thinking critically about this, we don't believe the date of observation should have any input on our model, especially since the unseen data we would like to make predictions for will not have observation dates listed.

In [11]:
# Drop date_recorded
df = df.drop('date_recorded', axis=1)
drop_columns.append('date_recorded')

#### funder and installer

In [12]:
df.funder.value_counts(dropna=False)

Government Of Tanzania      9084
NaN                         3635
Danida                      3114
Hesawa                      2202
Rwssp                       1374
                            ... 
Quick Win                      1
Frankfurt                      1
Ded_rwsp                       1
Dokta Mwandulam                1
Luchelegu Primary School       1
Name: funder, Length: 1898, dtype: int64

In [13]:
df.installer.value_counts(dropna=False)

DWE                         17402
NaN                          3655
Government                   1825
RWE                          1206
Commu                        1060
                            ...  
Moshono ADP                     1
VILLAGE WATER COMMISSION        1
Pet  Corporation  Ltd           1
Got                             1
OLOMOLOKI                       1
Name: installer, Length: 2146, dtype: int64

We've got way too many categories here to encode fully. Plus, all the single-instance categories wouldn't give us any significant predictive signal to analyze. Let's replace any category with less than 100 instances with 'Other'.

In [14]:
# Create list of funders with less than 100 appearances
low_count_funders = df.funder.value_counts()[df.funder.value_counts() < 100].index.tolist()

# Use mask function to replace funders in this list with 'Other'
df.funder.mask(df.funder.isin(low_count_funders), 'Other', inplace=True)

In [15]:
# Do the same for installers
low_count_installers = df.installer.value_counts()[df.installer.value_counts() < 100].index.tolist()

df.installer.mask(df.installer.isin(low_count_installers), 'Other', inplace=True)

In [16]:
df.funder.value_counts(dropna=False)

Other                            12379
Government Of Tanzania            9084
NaN                               3635
Danida                            3114
Hesawa                            2202
                                 ...  
Ru                                 105
Village Council                    105
Halmashauri Ya Wilaya Sikonge      102
Germany                            101
Hsw                                101
Name: funder, Length: 93, dtype: int64

In [17]:
df.installer.value_counts(dropna=False)

DWE                              17402
Other                            13545
NaN                               3655
Government                        1825
RWE                               1206
                                 ...  
FINI WATER                         103
WATER AID                          103
Halmashauri ya wilaya sikonge      102
Wizara ya maji                     100
HSW                                100
Name: installer, Length: 86, dtype: int64

Great! We're not sure if these will be useful, but maybe! Let's throw them in our categorical column list.

In [18]:
cat_columns.append('funder')
cat_columns.append('installer')

#### gps_height

In [19]:
df.gps_height.describe()

count    59400.000000
mean       668.297239
std        693.116350
min        -90.000000
25%          0.000000
50%        369.000000
75%       1319.250000
max       2770.000000
Name: gps_height, dtype: float64

In [20]:
df.gps_height.value_counts(dropna=False)

 0       20438
-15         60
-16         55
-13         55
-20         52
         ...  
 2285        1
 2424        1
 2552        1
 2413        1
 2385        1
Name: gps_height, Length: 2428, dtype: int64

A bit of the same issue as the tsh_amount column. Lot's of 0's. Let's replace with NaN's and throw it in our numeric group.

In [21]:
# Replace 0 with NaN
df = df.replace({'gps_height': {0: np.nan}})

# gps_height will go into numerical column list
num_columns.append('gps_height')

#### longitude, latitude

In [22]:
df.longitude.value_counts(dropna=False)

0.000000     1812
39.088875       2
39.105307       2
37.543401       2
38.180538       2
             ... 
38.710520       1
40.117029       1
34.672962       1
39.433604       1
34.890838       1
Name: longitude, Length: 57516, dtype: int64

In [23]:
df.latitude.value_counts(dropna=False)

-2.000000e-08    1812
-6.990549e+00       2
-2.489378e+00       2
-2.515321e+00       2
-6.963565e+00       2
                 ... 
-1.086099e+01       1
-3.149066e+00       1
-3.305834e+00       1
-9.076967e+00       1
-2.598965e+00       1
Name: latitude, Length: 57517, dtype: int64

We found some hidden nulls in longitude and latitude. Let's recode those as NaN's and then add these to the numeric columns. We will impute these later and create a missing indicator.

In [24]:
df = df.replace({'longitude': {0: np.nan},
            'latitude': {-2e-08: np.nan}})

In [25]:
# longitude and latitude will go into numerical column list
num_columns.append('longitude')
num_columns.append('latitude')

#### wpt_name

In [26]:
df.wpt_name.value_counts(dropna=False)

none                  3563
Shuleni               1748
Zahanati               830
Msikitini              535
Kanisani               323
                      ... 
Kwa Zambia               1
Kwa Ambali               1
Pundu                    1
Kwa Romans Kiluvya       1
Tujikongoje              1
Name: wpt_name, Length: 37400, dtype: int64

We'll drop this column as a name shouldn't have any predictive power. 

In [27]:
# Drop wpt_name
df = df.drop('wpt_name', axis=1)
drop_columns.append('wpt_name')

#### num_private

In [28]:
df.num_private.value_counts(dropna=False)

0      58643
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

The data dictionary doesn't have any info on this column, and it's extremely concentrated at zero anyways. Drop.

In [29]:
# Drop num_private
df = df.drop('num_private', axis=1)
drop_columns.append('num_private')

#### basin

In [30]:
df.basin.value_counts(dropna=False)

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64

A good categorical column. Let's add it to the list. We'll have to One Hot Encode this one. 

In [31]:
cat_columns.append('basin')

#### subvillage, region, region_code, district_code, lga, ward

According to the data dictionary, all of these features are for geographic location. We definitely want to use these, but let's pick the most useful looking ones.

In [32]:
df.subvillage.value_counts(dropna=False)

Madukani         508
Shuleni          506
Majengo          502
Kati             373
NaN              371
                ... 
Mfuni              1
Buloswa            1
Mamndikongo        1
Kihuka             1
Sanje Ya Kati      1
Name: subvillage, Length: 19288, dtype: int64

In [33]:
df.region.value_counts(dropna=False)

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [34]:
df.region_code.value_counts(dropna=False)

11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64

In [35]:
df.district_code.value_counts(dropna=False)

1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
30      995
33      874
53      745
43      505
13      391
23      293
63      195
62      109
60       63
0        23
80       12
67        6
Name: district_code, dtype: int64

In [36]:
df.lga.value_counts(dropna=False)

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 125, dtype: int64

In [37]:
df.ward.value_counts(dropna=False)

Igosi        307
Imalinyi     252
Siha Kati    232
Mdandu       231
Nduruma      217
            ... 
Uchindile      1
Mkumbi         1
Matarawe       1
Ifinga         1
Izia           1
Name: ward, Length: 2092, dtype: int64

Our first time through, we only picked 2 of these 6. We see now that we aren't even close to overfitting, so we're going to feed all of these to our model. First, let's replace any categories with less than 100 instances with other, then append all of them to the categorical columns.

In [38]:
# Create list of subvillages with less than 100 appearances
low_count_subvillage = df.subvillage.value_counts()[df.subvillage.value_counts() < 100].index.tolist()

# Use mask function to replace subvillages in this list with 'Other'
df.subvillage.mask(df.subvillage.isin(low_count_subvillage), 'Other', inplace=True)

# Same for lga and ward

low_count_lga = df.lga.value_counts()[df.lga.value_counts() < 100].index.tolist()
df.lga.mask(df.lga.isin(low_count_lga), 'Other', inplace=True)

low_count_ward = df.ward.value_counts()[df.ward.value_counts() < 100].index.tolist()
df.ward.mask(df.ward.isin(low_count_ward), 'Other', inplace=True)

In [39]:
cat_columns.append('subvillage')
cat_columns.append('region_code')
cat_columns.append('region')
cat_columns.append('district_code')
cat_columns.append('lga')
cat_columns.append('ward')

#### population

In [40]:
df.population.value_counts(dropna=False)

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

Again, a huge concentration of near-zero numbers. We don't buy that 21k wells have no one around. We also don't buy that 7k have only 1 person, for that matter. We're going to replace those with NaN's and put this in the numeric list.

In [41]:
df = df.replace({'population': {0: np.nan,
                               1: np.nan}})

num_columns.append('population')

#### public_meeting

In [42]:
df.public_meeting.value_counts(dropna=False)

True     51011
False     5055
NaN       3334
Name: public_meeting, dtype: int64

We're not entirely sure what this one is, but we feel comfortable leaving it in. This is a binary categorical classifier. We will want to treat this differently than our other categoricals because it doesn't need to be one hot encoded to be useful to our Logistic Regressor.

In [43]:
bin_columns.append('public_meeting')

#### recorded_by

In [44]:
df.recorded_by.value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

Only one value, not predictive!

In [45]:
# Drop recorded_by
df = df.drop('recorded_by', axis=1)
drop_columns.append('recorded_by')

#### scheme_management and scheme_name

The data dictionary describes both of these features as "who operates the waterpoint". Let's try and pick one.

In [46]:
df.scheme_management.value_counts(dropna=False)

VWC                 36793
WUG                  5206
NaN                  3877
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [47]:
df.scheme_name.value_counts(dropna=False)

NaN                         28166
K                             682
None                          644
Borehole                      546
Chalinze wate                 405
                            ...  
Kibaon                          1
Ka                              1
KAIBANJA PUMPING SCHEME         1
FinWater Borehole Scheme        1
Rain harvest                    1
Name: scheme_name, Length: 2697, dtype: int64

Okay, we're definitely going with scheme_management. That's a clean categorical. We should also replace the 'none' value with a NaN.

In [48]:
# Replace None in scheme_management with Nan
df = df.replace({'scheme_management': {'None': np.nan}})
cat_columns.append('scheme_management')
                 
# Drop scheme_name
df = df.drop('scheme_name', axis=1)
drop_columns.append('scheme_name')

#### Permit

In [49]:
df.permit.value_counts(dropna=False)

True     38852
False    17492
NaN       3056
Name: permit, dtype: int64

Binary!

In [50]:
bin_columns.append('permit')

#### construction_year

In [51]:
df.construction_year.value_counts(dropna=False)

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

This one is tricky for us as a beginner ML practitioner. Our first instinct is to treat it as numeric, but really it's categorical. Let's replace those 0's with NaN's and throw it in the categorical list.

In [52]:
df = df.replace({'construction_year': {0: np.nan}})
cat_columns.append('construction_year')

#### extraction_type, extraction_type_group, extraction_type_class

These look very similar to each other. The dictionary gives the same description for each as "the kind of extraction the waterpoint uses". Let's try and pick just one again.

In [53]:
df.extraction_type.value_counts(dropna=False)

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [54]:
df.extraction_type_group.value_counts(dropna=False)

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [55]:
df.extraction_type_class.value_counts(dropna=False)

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

Okay, so each one is a bit more granular than the other. The most granular has a few categories with too few values for us to be comfortable with. If we chose the least granular, we would be worried about potentially losing some important signals coming from the type of pump. Let's go with extraction_type_group and drop the other 2.

In [56]:
cat_columns.append('extraction_type_group')

df = df.drop('extraction_type', axis=1)
df = df.drop('extraction_type_class', axis=1)
drop_columns.append('extraction_type')
drop_columns.append('extraction_type_class')

#### management and management_group

These look very similar to our scheme columns. We may not need these at all.

In [57]:
df.management.value_counts(dropna=False)

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [58]:
df.management_group.value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

The management_group is less granular, so we know we would get a better signal from the management feature. But it looks very close to our scheme_management feature...

In [59]:
df.scheme_management.value_counts(dropna=False)

VWC                 36793
WUG                  5206
NaN                  3878
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
Name: scheme_management, dtype: int64

Close, but not the same. That makes sense when we think about it. There could be many wells operated by one group and managed by another. Let's drop management_group and keep management.

In [60]:
cat_columns.append('management')

df = df.drop('management_group', axis=1)
drop_columns.append('management_group')

#### payment and payment_type

In [61]:
df.payment.value_counts(dropna=False)

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [62]:
df.payment_type.value_counts(dropna=False)

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

Clear duplicates! Nice. We will keep payment_type because we like the slightly shorter string values given for the categories.

In [63]:
cat_columns.append('payment_type')

df = df.drop('payment', axis=1)
drop_columns.append('payment')

#### water_quality and quality_group

In [64]:
df.water_quality.value_counts(dropna=False)

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [65]:
df.quality_group.value_counts(dropna=False)

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

Let's go with water_quality for the extra level of granularity!

In [66]:
cat_columns.append('water_quality')

df = df.drop('quality_group', axis=1)
drop_columns.append('quality_group')

#### quantity and quantity_group

In [67]:
df.quantity.value_counts(dropna=False)

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [68]:
df.quantity_group.value_counts(dropna=False)

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

Copies! Let's go with quantity for the briefer name.

In [69]:
cat_columns.append('quantity')

df = df.drop('quantity_group', axis=1)
drop_columns.append('quantity_group')

#### source, source_type, source_class

Another group of features with the same description in the dictionary. Who will win?

In [70]:
df.source.value_counts(dropna=False)

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [71]:
df.source_type.value_counts(dropna=False)

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [72]:
df.source_class.value_counts(dropna=False)

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

We like the granularity given with the source feature. We can accept 66 in the least frequent class as it is an unknown, and we'd like to see lake/river broken out as well as machine/hand drilled wells.

In [73]:
cat_columns.append('source')

df = df.drop('source_type', axis=1)
df = df.drop('source_class', axis=1)
drop_columns.append('source_type')
drop_columns.append('source_class')

#### waterpoint_type and waterpoint_type_group

And last but not least! Dealer's choice.

In [74]:
df.waterpoint_type.value_counts(dropna=False)

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [75]:
df.waterpoint_type_group.value_counts(dropna=False)

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

A tiny bit more detail with waterpoint_type. Let's go with it.

In [76]:
cat_columns.append('waterpoint_type')

df = df.drop('waterpoint_type_group', axis=1)
drop_columns.append('waterpoint_type_group')

### What's left?

In [77]:
df.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type_group,management,payment_type,water_quality,quantity,source,waterpoint_type,status_group
0,6000.0,Roman,1390.0,Other,34.938093,-9.856322,Lake Nyasa,Other,Iringa,11,5,Ludewa,Other,109.0,True,VWC,False,1999.0,gravity,vwc,annually,soft,enough,spring,communal standpipe,1
1,NaN,Other,1399.0,Other,34.698766,-2.147466,Lake Victoria,Other,Mara,20,2,Serengeti,Other,280.0,NaN,Other,True,2010.0,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,1
2,25.0,Other,686.0,World vision,37.460664,-3.821329,Pangani,Majengo,Manyara,21,4,Simanjiro,Other,250.0,True,VWC,True,2009.0,gravity,vwc,per bucket,soft,enough,dam,communal standpipe multiple,1
3,NaN,Unicef,263.0,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Other,Mtwara,90,63,Nanyumbu,Other,58.0,True,VWC,True,1986.0,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,0
4,NaN,Other,NaN,Artisan,31.130847,-1.825359,Lake Victoria,Other,Kagera,18,1,Karagwe,Other,NaN,True,NaN,True,NaN,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,1


In [78]:
df_cleaned = df

So for our numeric columns, we will be scaling and creating some Null or 0 indicator categories.

In [79]:
num_columns

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population']

And we'll One Hot Encode all these categorical columns. We'll also create Null indicators here and use Simple Imputer on any NaN's.

In [80]:
cat_columns

['funder',
 'installer',
 'basin',
 'subvillage',
 'region_code',
 'region',
 'district_code',
 'lga',
 'ward',
 'scheme_management',
 'construction_year',
 'extraction_type_group',
 'management',
 'payment_type',
 'water_quality',
 'quantity',
 'source',
 'waterpoint_type']

We just need to impute these and they will be model-ready.

In [81]:
bin_columns

['public_meeting', 'permit']

And for easy reference later, here are the columns we dropped.

In [82]:
drop_columns

['date_recorded',
 'wpt_name',
 'num_private',
 'recorded_by',
 'scheme_name',
 'extraction_type',
 'extraction_type_class',
 'management_group',
 'payment',
 'quality_group',
 'quantity_group',
 'source_type',
 'source_class',
 'waterpoint_type_group']

#### Save Down

In [83]:
df_cleaned.to_csv('../data/Tanzania_Training_Cleaned.csv', index=False)